In this notebook we will evaluate the results of the pipeline

## The Subsampled test 
first let's do the test with the subsampled set (15 tasks)

In [1]:
from src.pipeline.evaluation import evaluation
from utils.task import Task
import json
import os
from dotenv import load_dotenv
from tqdm import tqdm

load_dotenv()

True

In [2]:
# Function to load JSON data
def load_json_data(filepath):
    with open(filepath, 'r') as file:
        data = json.load(file)
    return data


# Function to create task object
def create_task(example):
    return Task(example)

In [3]:
# load the task data
filepath = "C:/Users\yousf\Bureau\ConvergenceAI\CHESS_Impl\data/test/subsampled_test.json"
data = load_json_data(filepath)
# load the candidate SQL 
filepath_query = "C:/Users\yousf\Bureau\ConvergenceAI\CHESS_Impl\data/test/generated_candidate.json"
generated_candidate = load_json_data(filepath_query)
# load the revised SQL
filepath_query = "C:/Users\yousf\Bureau\ConvergenceAI\CHESS_Impl\data/test/final_sql.json"
result_sql = load_json_data(filepath_query)

In [4]:
res = []
for index, sample in enumerate(data):
    task = create_task(sample)
    ans = evaluation(task=task, generated_candidate=generated_candidate[index], revised_candidate=result_sql[index])
    res.append(ans)
    print(str(index) + " : ", ans)
    print("\n")

0 :  {'Question': 'List out the atom id that belongs to the TR346 molecule and how many bond type can be created by this molecule?', 'Evidence': '', 'GOLD_SQL': "SELECT T1.atom_id, COUNT(DISTINCT T2.bond_type) FROM atom AS T1 INNER JOIN bond AS T2 ON T1.molecule_id = T2.molecule_id WHERE T1.molecule_id = 'TR346' GROUP BY T1.atom_id, T2.bond_type", 'candidate_generation': {'exec_res': 0, 'exec_err': 'no such table: molecule_atoms', 'Predicted SQL': "SELECT T1.atom_id, COUNT(DISTINCT T2.bond_type) AS bond_type_count FROM molecule_atoms T1 JOIN bonds T2 ON T1.atom_id = T2.atom_id WHERE T1.molecule_id = 'TR346' AND T2.bond_type IS NOT NULL GROUP BY T1.molecule_id"}, 'revision': {'exec_res': 1, 'exec_err': '--', 'Predicted SQL': "SELECT T1.atom_id, COUNT(DISTINCT T2.bond_type) AS bond_type_count FROM atom AS T1 JOIN bond AS T2 ON T1.molecule_id = T2.molecule_id WHERE T1.molecule_id = 'TR346' AND T2.bond_type IS NOT NULL GROUP BY T1.atom_id"}}


1 :  {'Question': 'What is the city location o

In [8]:
percentage1 = 0
percentage2 = 0
for result in res:
    percentage1 += result['candidate_generation']['exec_res']
    percentage2 += result['revision']['exec_res']

print("The Execution accuracy from the candidate generation module: ", percentage1*100 / len(res),"%")
print("The Execution accuracy after revision: ", percentage2*100 / len(res),"%")

The Execution accuracy from the candidate generation module:  0.0 %
The Execution accuracy after revision:  60.0 %
